In [2]:
import glob
import pickle
import numpy
from tqdm import tqdm
!pip install music21
from music21 import converter, instrument, note, chord, stream
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import BatchNormalization as BatchNorm
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

  Using cached music21-6.7.1-py3-none-any.whl
  Using cached more_itertools-8.7.0-py3-none-any.whl (48 kB)
  Using cached webcolors-1.11.1-py3-none-any.whl (9.9 kB)


In [5]:
def train_network(paths):
    """ Train a Neural Network to generate music """
    notes = get_notes(paths)

    # get amount of pitch names
    n_vocab = len(set(notes))

    network_input, network_output = prepare_sequences(notes, n_vocab)

    model = create_network(network_input, n_vocab)

    train(model, network_input, network_output)

    return model

def get_notes(paths):
    """ Get all the notes and chords from the midi files in the ./midi_songs directory """
    notes = []

    for file in tqdm(paths):
        midi = converter.parse(file)

        # print("Parsing %s" % file)

        notes_to_parse = None

        try: # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[1].recurse() 
        except: # file has notes in a flat structure
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    with open('data/notes', 'wb') as filepath:
        pickle.dump(notes, filepath)

    return notes

def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 100

    # get all pitch names
    pitchnames = sorted(set(item for item in notes))

     # create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    network_input = network_input / float(n_vocab)

    network_output = np_utils.to_categorical(network_output)

    return (network_input, network_output)

def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))
    model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.3,))
    model.add(LSTM(512))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    return model

def train(model, network_input, network_output):
    """ train the neural network """
    filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
    checkpoint = ModelCheckpoint(
        filepath,
        monitor='loss',
        verbose=1,
        save_best_only=True,
        mode='min'
    )
    callbacks_list = [checkpoint]

    model.fit(network_input, network_output, epochs=10, batch_size=128, callbacks=callbacks_list)

In [11]:
paths = glob.glob(r"data\classical_composers\midi\*\*.mid")
paths = [path.replace('\\','/') for path in paths]

# paths

In [43]:
midi = converter.parse(paths[-3])
s2 = instrument.partitionByInstrument(midi)
len(s2.parts)

3

In [52]:
notes = []

notes_to_parse = s2.parts[1].recurse() 

for element in notes_to_parse:
    if isinstance(element, note.Note):
        # print(element)
        notes.append(str(element.pitch))
    elif isinstance(element, chord.Chord):
        print(element.normalOrder, element)
        notes.append('.'.join(str(n) for n in element.normalOrder))

[8, 11] <music21.chord.Chord G#4 B4>
[11, 4] <music21.chord.Chord B3 E3>
[8, 11] <music21.chord.Chord B4 G#4>
[11, 4] <music21.chord.Chord E3 B3>
[6, 9] <music21.chord.Chord F#4 A4>
[11, 4] <music21.chord.Chord B3 E3>
[4, 8] <music21.chord.Chord G#4 E4>
[4, 8] <music21.chord.Chord E3 G#3>
[8, 11] <music21.chord.Chord G#4 B4>
[11, 4] <music21.chord.Chord B3 E3>
[8, 11] <music21.chord.Chord G#4 B4>
[11, 4] <music21.chord.Chord E3 B3>
[11, 3] <music21.chord.Chord E-5 B4>
[8, 11, 3] <music21.chord.Chord G#3 E-3 B3>
[7, 10] <music21.chord.Chord B-4 G4>
[10, 3] <music21.chord.Chord B-3 E-3>
[3, 8] <music21.chord.Chord E-4 G#4>
[3, 8] <music21.chord.Chord G#2 E-3>
[11, 3] <music21.chord.Chord B3 E-4>
[3, 9] <music21.chord.Chord E-4 A4 E-5>
[6, 11] <music21.chord.Chord B2 B3 F#3>
[3, 4, 9] <music21.chord.Chord E4 A4 E-5>
[9, 1, 4] <music21.chord.Chord E3 C#3 A3>
[5, 11] <music21.chord.Chord B4 F4>
[2, 8] <music21.chord.Chord D3 G#3>
[6, 11] <music21.chord.Chord F#4 B4>
[3, 9] <music21.chord.Ch

In [54]:
len(notes)

1365

In [ ]:
def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 30

    # get all pitch names
    pitchnames = sorted(set(item for item in notes))

     # create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    network_input = network_input / float(n_vocab)

    network_output = np_utils.to_categorical(network_output)

    return (network_input, network_output)

In [83]:
network_input = []
network_output = []

for i in range(0, len(notes) - 30):
    sequence_in = notes[i:i + 30]
    sequence_out = notes[i + 30]
    network_input.append([note_to_int[char] for char in sequence_in])
    network_output.append(note_to_int[sequence_out])

network_output = np_utils.to_categorical(network_output)

In [85]:
network_output.shape


(1335, 96)

In [59]:
pitchnames = sorted(set(item for item in notes))
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
note_to_int

{'0.4': 0,
 '0.4.7': 1,
 '0.6': 2,
 '1': 3,
 '1.4.8': 4,
 '10': 5,
 '10.3': 6,
 '11': 7,
 '11.1.4.7': 8,
 '11.2.6': 9,
 '11.3': 10,
 '11.3.6': 11,
 '11.4': 12,
 '2.6.9': 13,
 '2.7': 14,
 '2.8': 15,
 '3': 16,
 '3.4.9': 17,
 '3.6.9': 18,
 '3.8': 19,
 '3.9': 20,
 '4': 21,
 '4.6.9': 22,
 '4.7.9': 23,
 '4.8': 24,
 '4.8.10': 25,
 '4.8.11': 26,
 '4.9': 27,
 '5.11': 28,
 '5.8.11': 29,
 '6': 30,
 '6.11': 31,
 '6.8.11': 32,
 '6.9': 33,
 '6.9.0': 34,
 '6.9.1': 35,
 '6.9.11': 36,
 '7.0': 37,
 '7.10': 38,
 '7.11': 39,
 '7.11.2': 40,
 '8.11': 41,
 '8.11.1': 42,
 '8.11.2': 43,
 '8.11.3': 44,
 '9.1.4': 45,
 '9.11.3': 46,
 '9.2': 47,
 'A2': 48,
 'A3': 49,
 'A4': 50,
 'A5': 51,
 'B-2': 52,
 'B-3': 53,
 'B-4': 54,
 'B-5': 55,
 'B2': 56,
 'B3': 57,
 'B4': 58,
 'B5': 59,
 'C#3': 60,
 'C#4': 61,
 'C#5': 62,
 'C#6': 63,
 'C5': 64,
 'C6': 65,
 'D2': 66,
 'D3': 67,
 'D4': 68,
 'D5': 69,
 'D6': 70,
 'E-3': 71,
 'E-4': 72,
 'E-5': 73,
 'E-6': 74,
 'E2': 75,
 'E3': 76,
 'E4': 77,
 'E5': 78,
 'E6': 79,
 'F#2': 80,

In [55]:
# notes = get_notes(['data/classical_composers/midi/beeth/elise.mid'])
# notes

In [10]:
train_network(paths[:15])

  7%|▋         | 1/15 [00:00<00:04,  3.15it/s]

Parsing ../input/classical-music-midi/brahms/br_im2.mid


 13%|█▎        | 2/15 [00:01<00:06,  1.88it/s]

Parsing ../input/classical-music-midi/brahms/br_im5.mid
Parsing ../input/classical-music-midi/brahms/brahms_opus1_4.mid


 20%|██        | 3/15 [00:02<00:13,  1.13s/it]

...

Parsing ../input/classical-music-midi/debussy/debussy_cc_2.mid
Parsing ../input/classical-music-midi/debussy/debussy_cc_1.mid


 93%|█████████▎| 14/15 [01:24<00:05,  5.68s/it]

Parsing ../input/classical-music-midi/debussy/debussy_cc_3.mid


100%|██████████| 15/15 [01:27<00:00,  5.83s/it]


Epoch 1/10
175/175 [==============================] - 116s 613ms/step - loss: 5.6408
Epoch 2/10
175/175 [==============================] - 107s 612ms/step - loss: 5.1639
Epoch 3/10
175/175 [==============================] - 108s 615ms/step - loss: 4.8768
Epoch 4/10
175/175 [==============================] - 107s 610ms/step - loss: 4.7822
Epoch 5/10
175/175 [==============================] - 106s 608ms/step - loss: 4.7439
Epoch 6/10
175/175 [==============================] - 107s 614ms/step - loss: 4.6960
Epoch 7/10
175/175 [==============================] - 106s 606ms/step - loss: 4.6590
Epoch 8/10
175/175 [==============================] - 105s 601ms/step - loss: 4.6115
Epoch 9/10
175/175 [==============================] - 107s 610ms/step - loss: 4.5668
Epoch 10/10
175/175 [==============================] - 106s 606ms/step - loss: 4.5446


In [24]:
def generate():
    """ Generate a piano midi file """
    #load the notes used to train the model
#     with open('data/notes', 'rb') as filepath:
#         notes = pickle.load(filepath)
    notes = get_notes(paths[:15])
    # Get all pitch names
    pitchnames = sorted(set(item for item in notes))
    # Get all pitch names
    n_vocab = len(set(notes))

    network_input, normalized_input = prepare_sequences(notes, pitchnames, n_vocab)
    model = create_network(normalized_input, n_vocab)
    prediction_output = generate_notes(model, network_input, pitchnames, n_vocab)
    create_midi(prediction_output)

def prepare_sequences(notes, pitchnames, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    # map between notes and integers and back
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    sequence_length = 100
    network_input = []
    output = []
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    normalized_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    normalized_input = normalized_input / float(n_vocab)

    return (network_input, normalized_input)

def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))
    model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.3,))
    model.add(LSTM(512))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    # Load the weights to each node
    model.load_weights('/kaggle/working/weights-improvement-01-5.4637-bigger.hdf5')

    return model

def generate_notes(model, network_input, pitchnames, n_vocab):
    """ Generate notes from the neural network based on a sequence of notes """
    # pick a random sequence from the input as a starting point for the prediction
    start = numpy.random.randint(0, len(network_input)-1)

    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

    pattern = network_input[start]
    prediction_output = []

    # generate 500 notes
    for note_index in range(500):
        prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)

        prediction = model.predict(prediction_input, verbose=0)

        index = numpy.argmax(prediction)
        result = int_to_note[index]
        prediction_output.append(result)

        pattern.append(index)
        pattern = pattern[1:len(pattern)]

    return prediction_output

def create_midi(prediction_output):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)

    midi_stream.write('midi', fp='test_output.mid')
    print('Output Saved')


In [25]:
generate() #./weights-improvement-10-4.5586-bigger.hdf5

  7%|▋         | 1/15 [00:00<00:04,  3.11it/s]

Parsing ../input/classical-music-midi/brahms/br_im2.mid


 13%|█▎        | 2/15 [00:00<00:03,  3.84it/s]

Parsing ../input/classical-music-midi/brahms/br_im5.mid
Parsing ../input/classical-music-midi/brahms/brahms_opus1_4.mid


 20%|██        | 3/15 [00:02<00:12,  1.02s/it]

...

 87%|████████▋ | 13/15 [00:12<00:01,  1.37it/s]

Parsing ../input/classical-music-midi/debussy/debussy_cc_2.mid
Parsing ../input/classical-music-midi/debussy/debussy_cc_1.mid


100%|██████████| 15/15 [00:13<00:00,  1.13it/s]

Parsing ../input/classical-music-midi/debussy/debussy_cc_3.mid


NameError: name 'stream' is not defined

In [23]:
glob.glob('/kaggle/working/*')

['/kaggle/working/weights-improvement-10-4.5586-bigger.hdf5',
 '/kaggle/working/weights-improvement-09-4.5891-bigger.hdf5',
 '/kaggle/working/__notebook_source__.ipynb',
 '/kaggle/working/weights-improvement-07-4.6603-bigger.hdf5',
 '/kaggle/working/weights-improvement-02-4.9959-bigger.hdf5',
 '/kaggle/working/weights-improvement-08-4.6218-bigger.hdf5',
 '/kaggle/working/weights-improvement-06-4.6926-bigger.hdf5',
 '/kaggle/working/weights-improvement-03-4.8240-bigger.hdf5',
 '/kaggle/working/weights-improvement-01-5.4637-bigger.hdf5',
 '/kaggle/working/weights-improvement-05-4.7203-bigger.hdf5',
 '/kaggle/working/weights-improvement-04-4.7614-bigger.hdf5']